In [27]:
# Autor: Efe Şirin
# Date: 2024-03-11

# IMPORTS

import os
import random
import torch
import random
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
import torch.optim as optim

In [28]:
# ======================= PYTHON SETTINGS ======================= #
# =======================   GPU or CPU    ======================= #
device = torch.device("cpu")
# Check if GPU is available -> CUDA
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))

# Apple Silicon GPU
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    out = torch.ones(1, device=mps_device)
    print (out)
    print ("MPS device found. - Apple Silicon GPU")
else:
    print ("MPS device not found.")

# =======================   Ranom Seeds   ======================= #
# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
# =============================================================== #

tensor([1.], device='mps:0')
MPS device found. - Apple Silicon GPU


In [29]:

# Define data directories
data_dir = 'Dataset'
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Load datasets
train_dataset = torchvision.datasets.ImageFolder(root=os.path.join(data_dir, train_dir), transform=transform)
fake_train_dataset = torchvision.datasets.ImageFolder(root= os.path.join(data_dir, train_dir), transform=transform)

val_dataset = torchvision.datasets.ImageFolder(root=f'{data_dir}/{val_dir}', transform=transform)
fake_val_dataset = torchvision.datasets.ImageFolder(root=f'{data_dir}/{val_dir}', transform=transform)

test_dataset = torchvision.datasets.ImageFolder(root=f'{data_dir}/{test_dir}', transform=transform)
fake_test_dataset = torchvision.datasets.ImageFolder(root=f'{data_dir}/{test_dir}', transform=transform)

# Combine real and fake datasets for training, validation, and testing
train_dataset = torch.utils.data.ConcatDataset([train_dataset, fake_train_dataset])
val_dataset = torch.utils.data.ConcatDataset([val_dataset, fake_val_dataset])
test_dataset = torch.utils.data.ConcatDataset([test_dataset, fake_test_dataset])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ! testing with subset of the data
# Define the portion of data you want to use
portion = 0.002  # for example, use 50% of the data

# Determine the size of the subset
subset_size_train = int(len(train_dataset) * portion)
subset_size_val = int(len(val_dataset) * portion)
subset_size_test = int(len(test_dataset) * portion)

# Create subsets of the datasets
train_dataset_subset, _ = torch.utils.data.random_split(train_dataset, [subset_size_train, len(train_dataset) - subset_size_train])
val_dataset_subset, _ = torch.utils.data.random_split(val_dataset, [subset_size_val, len(val_dataset) - subset_size_val])
test_dataset_subset, _ = torch.utils.data.random_split(test_dataset, [subset_size_test, len(test_dataset) - subset_size_test])

print(f"Size of the training subset: {len(train_dataset_subset)}")

# Create data loaders for the subsets
batch_size = 32
train_loader = DataLoader(train_dataset_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset_subset, batch_size=batch_size, shuffle=False)


Size of the training subset: 520


In [30]:

# Define the CNN model
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: fake and real

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [31]:
# ======================== GPU TRAINING ======================== #
# Initialize the model and move it to the GPU
model = CNNClassifier().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define training function
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        accuracy = correct / total
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {epoch_loss:.4f}, "
              f"Validation Loss: {val_loss:.4f}, "
              f"Validation Accuracy: {accuracy:.4f}")

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

# Evaluate the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {test_correct / test_total:.4f}")

# Save the model
torch.save(model.state_dict(), 'model.pth')
print("Model saved to model.pth")


Epoch 1/10, Train Loss: 1.6545, Validation Loss: 0.6879, Validation Accuracy: 0.5625
Epoch 2/10, Train Loss: 0.6935, Validation Loss: 0.6779, Validation Accuracy: 0.6250
Epoch 3/10, Train Loss: 0.6688, Validation Loss: 0.6472, Validation Accuracy: 0.6875
Epoch 4/10, Train Loss: 0.5896, Validation Loss: 0.6528, Validation Accuracy: 0.6250
Epoch 5/10, Train Loss: 0.4807, Validation Loss: 0.7152, Validation Accuracy: 0.6875
Epoch 6/10, Train Loss: 0.3206, Validation Loss: 0.8925, Validation Accuracy: 0.5312
Epoch 7/10, Train Loss: 0.2002, Validation Loss: 0.7250, Validation Accuracy: 0.7500
Epoch 8/10, Train Loss: 0.0926, Validation Loss: 0.9903, Validation Accuracy: 0.5000
Epoch 9/10, Train Loss: 0.0620, Validation Loss: 1.1070, Validation Accuracy: 0.5938
Epoch 10/10, Train Loss: 0.0186, Validation Loss: 1.1189, Validation Accuracy: 0.6250
Test Accuracy: 0.7258
Model saved to model.pth


In [32]:

# Initialize the model
model = CNNClassifier()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define training function
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        accuracy = correct / total
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {epoch_loss:.4f}, "
              f"Validation Loss: {val_loss:.4f}, "
              f"Validation Accuracy: {accuracy:.4f}")

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Epoch 1/10, Train Loss: 2.3184, Validation Loss: 0.6923, Validation Accuracy: 0.5000
Epoch 2/10, Train Loss: 0.6818, Validation Loss: 0.6617, Validation Accuracy: 0.5938
Epoch 3/10, Train Loss: 0.6509, Validation Loss: 0.6155, Validation Accuracy: 0.6562
Epoch 4/10, Train Loss: 0.5666, Validation Loss: 0.5534, Validation Accuracy: 0.7500
Epoch 5/10, Train Loss: 0.4595, Validation Loss: 0.5049, Validation Accuracy: 0.8125
Epoch 6/10, Train Loss: 0.3941, Validation Loss: 0.5946, Validation Accuracy: 0.7188
Epoch 7/10, Train Loss: 0.2737, Validation Loss: 0.5352, Validation Accuracy: 0.7500
Epoch 8/10, Train Loss: 0.1660, Validation Loss: 0.5458, Validation Accuracy: 0.8438
Epoch 9/10, Train Loss: 0.0754, Validation Loss: 0.5901, Validation Accuracy: 0.8438
Epoch 10/10, Train Loss: 0.0305, Validation Loss: 0.6871, Validation Accuracy: 0.8125


In [33]:

# Evaluate the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()


print(f"Test Accuracy: {test_correct / test_total:.4f}")

# Save the model
torch.save(model.state_dict(), 'model.pth')
print("Model saved to model.pth")


Test Accuracy: 0.6935
Model saved to model.pth


In [34]:
import torchvision.models as models

# Define the CNN model
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: fake and real

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define training function with transfer learning
def train_model_with_transfer_learning(backbone, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        backbone.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = backbone(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        backbone.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = backbone(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        accuracy = correct / total
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {epoch_loss:.4f}, "
              f"Validation Loss: {val_loss:.4f}, "
              f"Validation Accuracy: {accuracy:.4f}")

# Load a pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)

# Freeze all the parameters in the pre-trained model
for param in resnet18.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your problem
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 2)  # 2 classes: fake and real

# Create an instance of the transfer learning model with ResNet-18 backbone
transfer_model = resnet18

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transfer_model.parameters(), lr=0.001)

# Define data loaders (Assuming train_loader and val_loader are already defined)

# Train the model with transfer learning
train_model_with_transfer_learning(transfer_model, criterion, optimizer, train_loader, val_loader, num_epochs=10)



/Users/efe/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/efe/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Train Loss: 0.7527, Validation Loss: 0.6898, Validation Accuracy: 0.5312
Epoch 2/10, Train Loss: 0.6530, Validation Loss: 0.5950, Validation Accuracy: 0.7188
Epoch 3/10, Train Loss: 0.5889, Validation Loss: 0.5655, Validation Accuracy: 0.7500
Epoch 4/10, Train Loss: 0.5533, Validation Loss: 0.5397, Validation Accuracy: 0.7812
Epoch 5/10, Train Loss: 0.5228, Validation Loss: 0.5062, Validation Accuracy: 0.7812
Epoch 6/10, Train Loss: 0.4921, Validation Loss: 0.5002, Validation Accuracy: 0.8125
Epoch 7/10, Train Loss: 0.5076, Validation Loss: 0.4805, Validation Accuracy: 0.8125
Epoch 8/10, Train Loss: 0.5250, Validation Loss: 0.5071, Validation Accuracy: 0.7812
Epoch 9/10, Train Loss: 0.4627, Validation Loss: 0.4376, Validation Accuracy: 0.8125
Epoch 10/10, Train Loss: 0.4969, Validation Loss: 0.4404, Validation Accuracy: 0.8125


In [35]:

# Evaluate the model
transfer_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = transfer_model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {test_correct / test_total:.4f}")

# Save the model
torch.save(transfer_model.state_dict(), 'transfer_model.pth')

Test Accuracy: 0.8387
